<a href="https://colab.research.google.com/github/MUTIS-Ethical-Investment-Fund/Octave-Platform/blob/main/cost_of_capital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
!pip install yfinance

url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
links = [link.get("href") for link in soup.find_all("a")]
factor_links = [link for link in links if link and "5_factors_csv" in link.lower()]
momentum_links = [link for link in links if link and "mom_factor_csv" in link.lower()]

for link in factor_links:
  print(link)


ftp/Developed_5_Factors_CSV.zip
ftp/Developed_ex_US_5_Factors_CSV.zip
ftp/Europe_5_Factors_CSV.zip
ftp/Japan_5_Factors_CSV.zip
ftp/Asia_Pacific_ex_Japan_5_Factors_CSV.zip
ftp/North_America_5_Factors_CSV.zip
ftp/Emerging_5_Factors_CSV.zip


In [ ]:
for link in momentum_links:
  print(link)


ftp/Developed_Mom_Factor_CSV.zip
ftp/Developed_ex_US_Mom_Factor_CSV.zip
ftp/Europe_Mom_Factor_CSV.zip
ftp/Japan_Mom_Factor_CSV.zip
ftp/Asia_Pacific_ex_Japan_MOM_Factor_CSV.zip
ftp/North_America_Mom_Factor_CSV.zip
ftp/Emerging_MOM_Factor_CSV.zip


In [ ]:
import pandas as pd
import io
import zipfile

def download_zip(url, file_path):
  # Use the requests library to download the file
  print(url + "/" + file_path)
  response = requests.get(url + "/" + file_path)

  # Check if the request was successful
  if response.status_code == 200:
    # Open the zip file from the memory buffer
    zipfile_in_memory = io.BytesIO(response.content)
    with zipfile.ZipFile(zipfile_in_memory, "r") as zip_ref:
      # Extract the first file from the zip archive
      csv_filename = zip_ref.namelist()[0]
      with zip_ref.open(csv_filename) as csv_file:
        # Load the contents of the CSV file into a pandas DataFrame 
        # skip description rows
        df = pd.read_csv(csv_file, skiprows=6)
  else:
    # If the request was not successful, raise an error
    raise Exception("Failed to download the file")
 
  return df

In [ ]:
def clean_data(df):
    
  # Delete the annual Factors
  index_to_drop = df[df.iloc[:, 0] == " Annual Factors: January-December"].index
  df = df.iloc[:index_to_drop[0]]

  # Format the Months Field
  df = df.rename(columns={'Unnamed: 0': 'Date'})
  df['Date'] = df['Date'].str.strip()
  df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')

  # Take the last 60 months of data
  date_threshold = df['Date'].max() - pd.DateOffset(months=60)
  df = df[df['Date'] >= date_threshold]
  
  return df

In [ ]:
url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french"
file_path = factor_links[5]
factors_df = download_zip(url, file_path)
factors_df = clean_data(factors_df)
factors_df

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/North_America_5_Factors_CSV.zip


,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
330,2018-01-01,5.23,-2.88,-2.87,0.00,-1.08,0.11
331,2018-02-01,-3.95,-0.23,-2.74,0.86,-2.59,0.11
332,2018-03-01,-2.17,2.98,0.16,-0.46,-0.04,0.12
333,2018-04-01,0.35,0.46,1.36,-1.91,0.74,0.14
334,2018-05-01,2.51,2.86,-3.41,-1.37,-1.71,0.14
...,...,...,...,...,...,...,...
386,2022-09-01,-9.30,-0.82,0.55,-1.57,-0.05,0.19
387,2022-10-01,7.96,1.44,7.66,1.10,6.64,0.23
388,2022-11-01,4.82,-1.21,0.85,3.54,2.54,0.29
389,2022-12-01,-6.30,0.31,1.77,0.14,3.92,0.33


In [ ]:
file_path = momentum_links[5]
momentum_df = download_zip(url, file_path)
momentum_df = clean_data(momentum_df)
momentum_df

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/North_America_Mom_Factor_CSV.zip


,Date,WML
326,2018-01-01,3.92
327,2018-02-01,3.64
328,2018-03-01,-1.23
329,2018-04-01,0.51
330,2018-05-01,4.00
...,...,...
382,2022-09-01,3.48
383,2022-10-01,5.77
384,2022-11-01,-2.38
385,2022-12-01,4.02


In [ ]:
merged_df = factors_df.merge(momentum_df, on='Date')
merged_df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,WML
0,2018-01-01,5.23,-2.88,-2.87,0.00,-1.08,0.11,3.92
1,2018-02-01,-3.95,-0.23,-2.74,0.86,-2.59,0.11,3.64
2,2018-03-01,-2.17,2.98,0.16,-0.46,-0.04,0.12,-1.23
3,2018-04-01,0.35,0.46,1.36,-1.91,0.74,0.14,0.51
4,2018-05-01,2.51,2.86,-3.41,-1.37,-1.71,0.14,4.00
...,...,...,...,...,...,...,...,...
56,2022-09-01,-9.30,-0.82,0.55,-1.57,-0.05,0.19,3.48
57,2022-10-01,7.96,1.44,7.66,1.10,6.64,0.23,5.77
58,2022-11-01,4.82,-1.21,0.85,3.54,2.54,0.29,-2.38
59,2022-12-01,-6.30,0.31,1.77,0.14,3.92,0.33,4.02


In [ ]:
# Convert the datatype to float
df_columns = merged_df.columns.tolist()
df_columns.remove('Date')
merged_df[df_columns] = merged_df[df_columns].astype(float)
# Normalize the values to percentages
merged_df.loc[:, merged_df.columns != 'Date'] /= 100
merged_df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,WML
0,2018-01-01,0.0523,-0.0288,-0.0287,0.0000,-0.0108,0.0011,0.0392
1,2018-02-01,-0.0395,-0.0023,-0.0274,0.0086,-0.0259,0.0011,0.0364
2,2018-03-01,-0.0217,0.0298,0.0016,-0.0046,-0.0004,0.0012,-0.0123
3,2018-04-01,0.0035,0.0046,0.0136,-0.0191,0.0074,0.0014,0.0051
4,2018-05-01,0.0251,0.0286,-0.0341,-0.0137,-0.0171,0.0014,0.0400
...,...,...,...,...,...,...,...,...
56,2022-09-01,-0.0930,-0.0082,0.0055,-0.0157,-0.0005,0.0019,0.0348
57,2022-10-01,0.0796,0.0144,0.0766,0.0110,0.0664,0.0023,0.0577
58,2022-11-01,0.0482,-0.0121,0.0085,0.0354,0.0254,0.0029,-0.0238
59,2022-12-01,-0.0630,0.0031,0.0177,0.0014,0.0392,0.0033,0.0402


In [ ]:
merged_df['Date'].max().strftime('%Y-%m-%d')

'2023-01-01'

In [ ]:
import yfinance as yf

ticker = "AAPL"  # Example ticker

df = yf.download(ticker, start=merged_df['Date'].min() - pd.DateOffset(months=1), end=merged_df['Date'].max(), interval="1mo")

# The resulting dataframe 'df' will contain the monthly historical prices for the given ticker.
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-12-01,42.487499,44.299999,41.615002,42.307499,40.230198,2124735200
2018-01-01,42.540001,45.025002,41.174999,41.857498,39.802288,2638717600
2018-02-01,41.792500,45.154999,37.560001,44.529999,42.343563,3711577200
2018-03-01,44.634998,45.875000,41.235001,41.945000,40.048107,2854910800
2018-04-01,41.660000,44.735001,40.157501,41.314999,39.446602,2664617200
...,...,...,...,...,...,...
2022-08-01,161.009995,176.149994,157.139999,157.220001,156.741898,1510239600
2022-09-01,156.639999,164.259995,138.000000,138.199997,137.971115,2084722800
2022-10-01,138.210007,157.500000,134.369995,153.339996,153.086044,1868139700


In [ ]:
df['RI'] = df['Adj Close'].pct_change()
df = df.reset_index()

merged_df = merged_df.merge(df, on='Date')

df_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
merged_df.drop(df_columns, axis=1, inplace=True)

merged_df['RI-RF'] = merged_df['RI'] - merged_df['RF']

merged_df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,WML,RI,RI-RF
0,2018-01-01,0.0523,-0.0288,-0.0287,0.0000,-0.0108,0.0011,0.0392,-0.010637,-0.011737
1,2018-02-01,-0.0395,-0.0023,-0.0274,0.0086,-0.0259,0.0011,0.0364,0.063847,0.062747
2,2018-03-01,-0.0217,0.0298,0.0016,-0.0046,-0.0004,0.0012,-0.0123,-0.054210,-0.055410
3,2018-04-01,0.0035,0.0046,0.0136,-0.0191,0.0074,0.0014,0.0051,-0.015020,-0.016420
4,2018-05-01,0.0251,0.0286,-0.0341,-0.0137,-0.0171,0.0014,0.0400,0.130763,0.129363
5,2018-06-01,0.0039,0.0018,-0.0104,0.0118,0.0088,0.0014,-0.0130,-0.005598,-0.006998
6,2018-07-01,0.0311,-0.0211,0.0079,0.0112,0.0055,0.0016,-0.0130,0.027983,0.026383
7,2018-08-01,0.0309,0.0056,-0.0577,0.0056,-0.0346,0.0016,0.0504,0.196227,0.194627
8,2018-09-01,0.0006,-0.0224,-0.0110,0.0038,0.0059,0.0015,-0.0009,-0.004825,-0.006325
9,2018-10-01,-0.0781,-0.0380,0.0367,0.0024,0.0282,0.0019,-0.0233,-0.030477,-0.032377


In [ ]:
merged_df.columns

Index(['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'WML', 'RI',
       'RI-RF'],
      dtype='object')

In [ ]:
import statsmodels.api as sm

y = merged_df['RI-RF']
X = merged_df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  RI-RF   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     17.70
Date:                Fri, 10 Feb 2023   Prob (F-statistic):           2.38e-10
Time:                        02:08:18   Log-Likelihood:                 86.654
No. Observations:                  60   AIC:                            -161.3
Df Residuals:                      54   BIC:                            -148.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0078      0.008      0.932      0.3

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
import statsmodels.api as sm

# Add a constant column to the independent variables
X = sm.add_constant(merged_df[['Mkt-RF', 'SMB', 'HML']])

# Fit the linear regression model
model = sm.OLS(merged_df['RI-RF'], X).fit()

# Get the regression coefficients
Alpha = model.params[0]
Beta_Mkt = model.params[1]
Beta_SMB = model.params[2]
Beta_HML = model.params[3]
Rf = merged_df['RF'].mean()

# Print the results
print("Alpha: ", Alpha)
print("Beta Mkt: ", Beta_Mkt)
print("Beta SMB: ", Beta_SMB)
print("Beta HML: ", Beta_HML)
print("Rf (Monthly): ", Rf)

Alpha:  0.012479782196476658
Beta Mkt:  1.2336152253352588
Beta SMB:  -0.40332070860869873
Beta HML:  -0.5588715491135652
Rf (Monthly):  0.0009616666666666667


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
